## Part 5: AgentCore Deployment and Testing

**Prerequisites**: Complete Part 2 first to have all tools and agent ready.

In [ ]:
# Import required libraries (if starting Part 5 fresh)
import json
import boto3
import uuid
from datetime import datetime

print("✅ Part 5 imports loaded!")
print("📝 Make sure you've run Part 2 to create the agent and tools")

## Save Complete Agent Code for Deployment

Let's save our complete agent code to a Python file for AgentCore deployment.

In [ ]:
# Complete agent code for deployment
agent_code = '''import json
import urllib3
import base64
import os
from datetime import datetime
from strands import Agent, tool
from bedrock_agentcore.runtime import BedrockAgentCoreApp

# SAP Configuration
SAP_CONFIG = {
    "base_url": "https://vhcals4hci.awspoc.club/sap/opu/odata/sap/API_SALES_ORDER_SRV/",
    "username": "gyanmis",
    "password": "Pass2025$",
    "client": "100"
}

def make_sap_request(entity_name, params=None):
    """Make SAP OData request with error handling"""
    try:
        http = urllib3.PoolManager()
        auth_b64 = base64.b64encode(f"{SAP_CONFIG[\"username\"]}:{SAP_CONFIG[\"password\"]}".encode("ascii")).decode("ascii")
        
        query_parts = []
        if params:
            for key, value in params.items():
                if value:
                    query_parts.append(f"${key}={value}")
        
        query_string = "&".join(query_parts)
        url = f"{SAP_CONFIG[\"base_url\"]}{entity_name}"
        if query_string:
            url += f"?{query_string}"
        
        headers = {
            "Authorization": f"Basic {auth_b64}",
            "Accept": "application/json",
            "sap-client": SAP_CONFIG["client"]
        }
        
        response = http.request("GET", url, headers=headers)
        
        if response.status == 200:
            return json.loads(response.data.decode("utf-8"))
        else:
            return {"error": f"HTTP {response.status}: {response.data.decode(\"utf-8\")}"}
            
    except Exception as e:
        return {"error": f"Request failed: {str(e)}"}

@tool
def get_sales_order_details(order_number: str) -> str:
    """Get complete details for a specific sales order number"""
    params = {"filter": f"SalesOrder eq \'{order_number}\""}
    result = make_sap_request("A_SalesOrder", params)
    
    if "error" in result:
        return f"Error: {result[\"error\"]}"
    
    records = result.get("d", {}).get("results", [])
    if not records:
        return f"Sales order {order_number} not found"
    
    order = records[0]
    response = f"""Order {order_number} Details:
• Order: {order.get("SalesOrder", "N/A")}
• Customer: {order.get("SoldToParty", "N/A")}
• Amount: {order.get("TotalNetAmount", "N/A")} {order.get("TransactionCurrency", "USD")}
• Type: {order.get("SalesOrderType", "N/A")}
• Status: {order.get("OverallSDProcessStatus", "N/A")}
• Date: {order.get("SalesOrderDate", "N/A")}"""
    
    if order.get("DeliveryBlockReason"):
        response += f"\\n🚫 Delivery Block: {order.get(\"DeliveryBlockReason\", \"N/A\")}"
    
    return response

@tool
def get_orders_with_delivery_blocks(limit: int = 10) -> str:
    """Get sales orders that have delivery blocks"""
    params = {
        "filter": "DeliveryBlockReason ne \"\"",
        "top": str(limit),
        "orderby": "SalesOrderDate desc"
    }
    result = make_sap_request("A_SalesOrder", params)
    
    if "error" in result:
        return f"Error: {result[\"error\"]}"
    
    records = result.get("d", {}).get("results", [])
    if not records:
        return "No orders found with delivery blocks"
    
    response = "Orders with Delivery Blocks:\\n"
    for order in records:
        response += f"• Order: {order.get(\"SalesOrder\", \"N/A\")}\\n"
        response += f"  Customer: {order.get(\"SoldToParty\", \"N/A\")}\\n"
        response += f"  Amount: {order.get(\"TotalNetAmount\", \"N/A\")} {order.get(\"TransactionCurrency\", \"USD\")}\\n"
        response += f"  🚫 Block Reason: {order.get(\"DeliveryBlockReason\", \"N/A\")}\\n\\n"
    
    return response

# Create agent with tools
agent = Agent(
    name="Enhanced SAP Sales Agent",
    description="SAP sales order assistant with delivery block support"
)

agent.tools = [get_sales_order_details, get_orders_with_delivery_blocks]

# AgentCore app
app = BedrockAgentCoreApp()

@app.entrypoint
def handler(payload):
    return {"response": "Agent response"}

if __name__ == "__main__":
    app.run()
'''

# Save complete agent code
with open('sap_strands_complete.py', 'w') as f:
    f.write(agent_code)

# Create requirements.txt
requirements = """strands
bedrock-agentcore
urllib3"""

with open('requirements.txt', 'w') as f:
    f.write(requirements)

print("✅ Complete agent code saved to 'sap_strands_complete.py'")
print("✅ Requirements.txt created")
print("📁 Files ready for AgentCore deployment")

## AgentCore Deployment Commands

Run these commands in your terminal to deploy the agent.

In [ ]:
# Display deployment commands
deployment_commands = """
# Terminal Commands for AgentCore Deployment:

# 1. Configure the agent
agentcore configure --entrypoint sap_strands_complete.py

# 2. Deploy to AgentCore
agentcore launch

# 3. Check deployment status
agentcore status

# 4. Test the deployed agent
agentcore invoke '{"prompt": "show me order 4062"}'

# 5. Test delivery blocks
agentcore invoke '{"prompt": "show me orders with delivery blocks"}'
"""

print("🚀 AgentCore Deployment Commands:")
print(deployment_commands)

## Python API Client for Deployed Agent

Access your deployed agent programmatically.

In [ ]:
def call_deployed_sap_agent(prompt, agent_arn):
    """
    Call your deployed SAP agent via AgentCore API
    
    Args:
        prompt (str): User query for the SAP agent
        agent_arn (str): ARN of your deployed agent
    
    Returns:
        str: Agent response
    """
    try:
        client = boto3.client('bedrock-agentcore', region_name='us-east-1')
        session_id = f"session-{uuid.uuid4().hex}"
        
        payload = {
            'prompt': prompt,
            'session_id': session_id
        }
        
        print(f"📤 Sending: {prompt}")
        
        response = client.invoke_agent_runtime(
            agentRuntimeArn=agent_arn,
            runtimeSessionId=session_id,
            payload=json.dumps(payload),
            qualifier="DEFAULT"
        )
        
        response_body = response['response'].read()
        response_data = json.loads(response_body)
        
        return response_data.get('response', 'No response received')
        
    except Exception as e:
        return f"Error calling agent: {str(e)}"

# Example usage (replace with your actual agent ARN after deployment)
EXAMPLE_AGENT_ARN = "arn:aws:bedrock-agentcore:us-east-1:123456789012:runtime/sap-agent-xyz"

print("✅ Agent client function created!")
print("🔗 Replace EXAMPLE_AGENT_ARN with your deployed agent's ARN")
print("\n📋 Sample usage:")
print('response = call_deployed_sap_agent("show me order 4062", YOUR_AGENT_ARN)')

## Sample Test Prompts

Test your deployed agent with these SAP queries.

In [ ]:
# Sample prompts for testing your deployed SAP agent
sample_prompts = [
    "show me order 4062",
    "show me value of order 5051", 
    "find orders with delivery blocks",
    "show me recent orders",
    "find orders for customer USCU_L09",
    "show me blocked orders",
    "what is the status of order 1749"
]

print("🧪 Sample Prompts for Testing Your Deployed Agent:")
print("=" * 60)

for i, prompt in enumerate(sample_prompts, 1):
    print(f"{i}. {prompt}")

print("\n" + "=" * 60)
print("💡 Testing Tips:")
print("• Start with specific order numbers (4062, 5051, 1749)")
print("• Test delivery block detection with 'blocked orders'")
print("• Try customer-specific queries")
print("• Observe which tools the agent selects for each query")
print("• Check CloudWatch logs for detailed execution traces")

## 🎉 Tutorial Complete!

### What You've Built:
- **🔧 5 SAP Strands Tools** for comprehensive order management
- **🚫 Delivery Block Detection** with business-critical filtering
- **🤖 Intelligent Agent** with automatic tool selection
- **🚀 AgentCore Deployment** for production scalability
- **📱 Python API Client** for programmatic access

### Next Steps:
1. **Deploy** using the terminal commands above
2. **Test** with the sample prompts
3. **Integrate** into your applications
4. **Monitor** via CloudWatch and AgentCore observability
5. **Extend** with additional SAP entities and business logic

**Congratulations on building a production-ready SAP AI agent! 🚀**